In [ ]:
#!pip install transformers
#!pip install --upgrade unsloth unsloth_zoo

In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM

import time
import os

#os.environ["PYTORCH_CUDA_ALLOC_CONF = expandable_segments:True"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

#!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
# --- 1. Configuration ---
# You can adjust these parameters
MODEL_NAME = "gpt2-xl"
MAX_LENGTH = 1024 # Maximum sequence length for GPT-2 Medium
BATCH_SIZE = 4
LEARNING_RATE = 5e-5
EPOCHS = 1
NUM_WORKERS = 4 # For faster data loading
PATH= 'MAAI_gpt2_extra_large_torch-50'

# --- 2. Device Setup (Essential for PyTorch) ---
# Automatically detects and uses GPU (CUDA/ROCm) or falls back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch using device: {device}")


/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


PyTorch using device: cuda


In [ ]:
!wget --no-check-certificate "https://jpdata1-my.sharepoint.com/:u:/g/personal/anand_jpdata_co/ERO3ZGvt96tIpqXOqhlfCFcBO4PksOLSKMtMFoQKX3BmCw?e=zM2QDz&download=1" -O MAAI_gpt2_large_torch_50

In [2]:

# --- 5. Initialization ---

# Load the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Set the padding token, which is necessary for batching
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the Model and move it to the device (GPU or CPU)
#model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Use the lines below for loading stored weights and comment the line above
model = torch.load(PATH, weights_only=False)



2025-10-09 13:43:02.302074: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-09 13:43:02.302709: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-09 13:43:02.305551: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-09 13:43:02.311503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760017382.321728   40440 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760017382.32

In [3]:
prompt = "तुमच्या"
max_new_tokens = 1024

# 1. Tokenize input and move to device
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)



In [4]:
# 2. Generate text
print(f"Generating continuation for prompt: '{prompt}'")
        # Use model.generate() for text generation
with torch.no_grad():
    output_sequences = model.generate(
    input_ids=input_ids,
    max_length=512, # Set max total length
    num_return_sequences=1,
    compile_config=None,
    do_sample=True, # Use sampling for more creative output
    top_k=50,
    top_p=0.95,
    temperature=0.9,
    pad_token_id=tokenizer.eos_token_id # Ensure generation stops correctly
    )

# 3. Decode and Print Output
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print("-" * 20)
print(f"Generated Text:\n{generated_text}")
print("-" * 20)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating continuation for prompt: 'तुमच्या'


/opt/conda/envs/py_3.12/lib/python3.12/site-packages/transformers/integrations/sdpa_attention.py:83: UserWarning: Using AOTriton backend for Efficient Attention forward... (Triggered internally at /var/lib/jenkins/pytorch/aten/src/ATen/native/transformers/hip/attention.hip:1180.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


--------------------
Generated Text:
तुमच्या ताकदीवर अनेक संकटं पेलून उभे राहिले आहे असे म्हणत असताना वैद्यकीय शिक्षणमंत्री हसन मुश्रीफ यांना अश्रू अनावर झाले. कोल्हापूरच्या कागलमधील सागांव येथे सोमवारी (दि. २३) सायंकाळी पुण्यातील जोजी नितीवेद येथे भाषण करण्यात आले. यावेळी त्यांनी एका मराठी तरुणाच्यावतीने शेतकऱ्यांवर आणि शेतकरी विविध रोषणाचे संकेत दिले
--------------------
